In [1]:
!pip install azure-ai-ml

  Using cached azure_ai_ml-1.22.4-py3-none-any.whl (12.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.5/286.5 kB 4.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.4/258.4 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: azure-storage-blob
    Found existing installation: azure-storage-blob 12.13.0
    Uninstalling azure-storage-blob-12.13.0:
      Successfully uninstalled azure-storage-blob-12.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-mlflow 1.57.0 requires azure-storage-blob<=12.19.0,>=12.

In [4]:
from azureml.core import Workspace, Dataset
from azureml.core.compute import ComputeTarget

# Connect to Azure ML Workspace
ws = Workspace.from_config()


In [5]:

# Define variables
endpoint_name = "svm-batch-ep"  # Replace with your endpoint name
input_data_path = "../data/energy_requirement_dataset.csv"    # Replace with your input data file path
output_data_path = "../data/output"     # Location to save output
compute_target_name = "computeclusterml" # Replace with your compute cluster name

In [9]:

# Upload input data to a datastore
datastore = ws.get_default_datastore()
'''
datastore.upload_files(
    files=[input_data_path],
    target_path="batch_inputs/",
    overwrite=True,
)
'''
input_data = Dataset.File.from_files(path=(datastore, "../data/energy_requirement_dataset.csv"))

In [10]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

# Set up Azure ML client
subscription_id = "9c15fe3b-278f-4875-b39a-d756406c9775"  # Replace with your Azure subscription ID
resource_group = "DP-100"    # Replace with your resource group name
workspace_name = "dp_100_learning"    # Replace with your Azure ML workspace name

# Authenticate and initialize MLClient
credential = DefaultAzureCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

# List all batch endpoints
batch_endpoints = ml_client.batch_endpoints.list()

# Print batch endpoint names
print("List of Batch Endpoints:")
for endpoint in batch_endpoints:
    print(f"Name: {endpoint.name}, Description: {endpoint.description}")


List of Batch Endpoints:
Name: svm-batch-ep, Description: None


In [4]:

# Get the batch endpoint
endpoint = PipelineEndpoint.get(workspace=ws, name=endpoint_name)

# Submit a batch job
job_name = "batch-prediction-job"
batch_job = endpoint.submit(
    input_datasets=[input_data],
    arguments=[
        "--input_data", "batch_inputs/input_data.csv",
        "--output_path", f"azureml://datastores/{datastore.name}/paths/{output_data_path}/"
    ],
    compute_target=ComputeTarget(workspace=ws, name=compute_target_name),
    name=job_name,
)

print(f"Batch job '{job_name}' submitted. Waiting for completion...")

# Wait for job completion
batch_job.wait_for_completion(show_output=True)

# Download results
batch_job.download(output_data_path)
print(f"Results downloaded to: {output_data_path}")

ErrorResponseException: (BadRequest) Response status code does not indicate success: 400 (Bad Request).
Microsoft.RelInfra.Common.Exceptions.ErrorResponseException: PipelineEndpoint name svm-batch-ep not found in workspace 62880345-ed91-473e-8c36-060e4852e575